In [1]:
from skimage.io import imread
import numpy as np
import fnmatch
import os
import re
import random

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.utils import Sequence, to_categorical

import sys
if "E:\\git\\keras-resnet" not in sys.path:
    sys.path.append("E:\\git\\keras-resnet")
    print(sys.path)
    
import resnet



['', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\keras-resnet']


Using TensorFlow backend.


In [2]:
#TODO check shuffle
class CustomDataGenerator(Sequence):

    def __init__(self, directory, batch_size, classes):
        self.directory = directory
        self.batch_size = batch_size
        self.classes = classes
        
        #create filenames, labels
        dirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
        print(dirs)
        self.files = []
        for d in dirs:
            fullpath_dir = os.path.join(directory,d)
            new_elements = [(f,d) for f in os.listdir(fullpath_dir)]
            self.files.extend(new_elements)
        
        # shuffle
        random.shuffle(self.files)
        print(len(self.files))
    
    
    def getLabels(self, offsetStr, viewStr, altitudeStr): 
        offset = 0
        view = 0
        altitude = 0
        for i, e in enumerate(self.classes[0]):
            if e == offsetStr:
                offset = i
                break
            
        for i, e in enumerate(self.classes[1]):
            if e == viewStr:
                view = i
                break
        
        for i, e in enumerate(self.classes[2]):
            if e == altitudeStr:
                altitude = i
                break
                
        return offset, view, altitude
    
    
    def __len__(self):
        return int(np.ceil(len(self.files) / float(self.batch_size)))

    def on_epoch_end(self):
        'Shuffle for the next epoch'
        random.shuffle(self.files)
    
    def __getitem__(self, idx):
        batch_x = []
        batch_y1 = []
        batch_y2 = []
        batch_y3 = []
        batch = self.files[idx * self.batch_size : (idx + 1) * self.batch_size]
        #print(self.directory)
        #print ('files from getitem(): ', batch)
        p = re.compile('(left|center|right)Offset_(left|straight|right)View_(low|medium|high)Altitude')
        
        for (file, d) in batch:
            
            finds = p.findall(d)
            #TODO assert only finds has only 1 element [(_,_)]
            (offsetStr, viewStr, altitudeStr) =  finds[0]
            (offset, view, altitude) = self.getLabels(offsetStr, viewStr, altitudeStr)
            
            folder = os.path.join(self.directory, d)
            rgba = imread(os.path.join(folder, file))
            rgb = rgba[:,:,:3]
            
            input_array = rgb / 255.0
            
            batch_x.append(input_array)
            batch_y1.append(offset)
            batch_y2.append(view)
            batch_y3.append(altitude)
         
        return np.array(batch_x), [np.array(to_categorical(batch_y1, 3)), np.array(to_categorical(batch_y2, 3)), np.array(to_categorical(batch_y3, 3))]


In [3]:
#sanity test CustomDataGenerator
# directory = directory = "E:\\UAV_drone\\data\\training\\lateral_and_view_normal\\train"
# classes = [['left', 'center', 'right'], ['left', 'straight', 'right']]
# datagen = CustomDataGenerator(directory, 5, classes)
# datagen.__getitem__(0)


In [ ]:
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'E:\\UAV_drone\\data\\training\\lateral_and_view_normal2_constant_distance_to_lines\\train'
val_data_dir = 'E:\\UAV_drone\\data\\training\\lateral_and_view_normal2_constant_distance_to_lines\\val'
nb_train_samples = 101687
nb_validation_samples = 25815
epochs = 100
batch_size = 32
img_channels = 3
nb_classes = 3

if K.image_data_format() == 'channels_first':
    input_shape = (img_channels, img_width, img_height)
else:
    input_shape = (img_width, img_height, img_channels)

# this is the augmentation configuration we will use for training
classes = [['left', 'center', 'right'], ['left', 'straight', 'right'], ['low', 'medium', 'high']]
train_datagen = CustomDataGenerator(train_data_dir, batch_size, classes)

# this is the augmentation configuration we will use for testing:
val_datagen = CustomDataGenerator(val_data_dir, batch_size, classes)


# checkpoint
current_directory = os.getcwd()
checkpoint_dir = os.path.join(current_directory, 'checkpoint')
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

file="model-uav-view_offset_altitude-normal-{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)


In [ ]:
model = resnet.ResnetBuilder.build_resnet_18_uav((img_channels, img_width, img_height), nb_classes, multi_output=3)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
model.fit_generator(
    train_datagen,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_datagen,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint])

Epoch 1/100
3177/3177 [==============================] - 4999s 2s/step - loss: 1.4993 - dense_1_loss: 0.4790 - dense_2_loss: 0.2915 - dense_3_loss: 0.3797 - dense_1_acc: 0.7869 - dense_2_acc: 0.8785 - dense_3_acc: 0.8265 - val_loss: 1.1416 - val_dense_1_loss: 0.5070 - val_dense_2_loss: 0.1966 - val_dense_3_loss: 0.1645 - val_dense_1_acc: 0.8019 - val_dense_2_acc: 0.9360 - val_dense_3_acc: 0.9379

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras-resnet\checkpoint\model-uav-view_offset_altitude-normal-01.hdf5
Epoch 2/100
3177/3177 [==============================] - 4794s 2s/step - loss: 0.6310 - dense_1_loss: 0.1978 - dense_2_loss: 0.0886 - dense_3_loss: 0.1079 - dense_1_acc: 0.9262 - dense_2_acc: 0.9723 - dense_3_acc: 0.9585 - val_loss: 1.7381 - val_dense_1_loss: 0.6261 - val_dense_2_loss: 0.4019 - val_dense_3_loss: 0.4967 - val_dense_1_acc: 0.7601 - val_dense_2_acc: 0.8581 - val_dense_3_acc: 0.8017

Epoch 00002: saving model to C:\Users\hoang\WorkingSpace\T

InternalError: GPU sync failed

## continue training if necessary

In [5]:
from keras.models import load_model
model_file = 'model-uav-view_offset_altitude-normal-09.hdf5'
model_full_path = os.path.join(checkpoint_dir, model_file)
model = load_model(model_full_path)

model.fit_generator(
    train_datagen,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_datagen,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint])

Epoch 1/100
3177/3177 [==============================] - 5675s 2s/step - loss: 0.2608 - dense_1_loss: 0.0598 - dense_2_loss: 0.0395 - dense_3_loss: 0.0296 - dense_1_acc: 0.9782 - dense_2_acc: 0.9863 - dense_3_acc: 0.9889 - val_loss: 0.9948 - val_dense_1_loss: 0.3542 - val_dense_2_loss: 0.1577 - val_dense_3_loss: 0.3546 - val_dense_1_acc: 0.9027 - val_dense_2_acc: 0.9518 - val_dense_3_acc: 0.9050

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras-resnet\checkpoint\model-uav-view_offset_altitude-normal-01.hdf5
Epoch 2/100
3177/3177 [==============================] - 4909s 2s/step - loss: 0.2524 - dense_1_loss: 0.0575 - dense_2_loss: 0.0371 - dense_3_loss: 0.0288 - dense_1_acc: 0.9793 - dense_2_acc: 0.9870 - dense_3_acc: 0.9892 - val_loss: 0.9667 - val_dense_1_loss: 0.4259 - val_dense_2_loss: 0.2321 - val_dense_3_loss: 0.1800 - val_dense_1_acc: 0.8704 - val_dense_2_acc: 0.9178 - val_dense_3_acc: 0.9300

Epoch 00002: saving model to C:\Users\hoang\WorkingSpace\T

3177/3177 [==============================] - 5023s 2s/step - loss: 0.1869 - dense_1_loss: 0.0378 - dense_2_loss: 0.0220 - dense_3_loss: 0.0180 - dense_1_acc: 0.9863 - dense_2_acc: 0.9927 - dense_3_acc: 0.9933 - val_loss: 0.9777 - val_dense_1_loss: 0.3899 - val_dense_2_loss: 0.3668 - val_dense_3_loss: 0.1133 - val_dense_1_acc: 0.8915 - val_dense_2_acc: 0.8838 - val_dense_3_acc: 0.9602

Epoch 00016: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras-resnet\checkpoint\model-uav-view_offset_altitude-normal-16.hdf5
Epoch 17/100
3177/3177 [==============================] - 5097s 2s/step - loss: 0.1872 - dense_1_loss: 0.0392 - dense_2_loss: 0.0220 - dense_3_loss: 0.0179 - dense_1_acc: 0.9860 - dense_2_acc: 0.9924 - dense_3_acc: 0.9932 - val_loss: 0.6386 - val_dense_1_loss: 0.2943 - val_dense_2_loss: 0.1665 - val_dense_3_loss: 0.0697 - val_dense_1_acc: 0.9082 - val_dense_2_acc: 0.9465 - val_dense_3_acc: 0.9763

Epoch 00017: saving model to C:\Users\hoang\WorkingSpace\TrainingMode

KeyboardInterrupt: 